In [1]:
from ngraph.scenario import Scenario
from ngraph.traffic_demand import TrafficDemand
from ngraph.traffic_manager import TrafficManager
from ngraph.lib.flow_policy import FlowPolicyConfig

In [2]:
scenario_yaml = """
blueprints:
  brick_2tier:
    groups:
      t1:
        node_count: 4
        name_template: t1-{node_num}
      t2:
        node_count: 4
        name_template: t2-{node_num}

    adjacency:
      - source: /t1
        target: /t2
        pattern: mesh
        link_params:
          capacity: 2
          cost: 1

  3tier_clos:
    groups:
      b1:
        use_blueprint: brick_2tier
      b2:
        use_blueprint: brick_2tier
      spine:
        node_count: 16
        name_template: t3-{node_num}

    adjacency:
      - source: b1/t2
        target: spine
        pattern: one_to_one
        link_params:
          capacity: 2
          cost: 1
      - source: b2/t2
        target: spine
        pattern: one_to_one
        link_params:
          capacity: 2
          cost: 1

network:
  name: "3tier_clos"
  version: 1.0

  groups:
    my_clos1:
      use_blueprint: 3tier_clos
"""
scenario = Scenario.from_yaml(scenario_yaml)
network = scenario.network

In [3]:
network.max_flow(
    source_path=r".*(b[0-9]*)/t1",
    sink_path=r".*(b[0-9]*)/t1",
    mode="pairwise",
    shortest_path=True,
)

{('b1', 'b1'): inf, ('b1', 'b2'): 32.0, ('b2', 'b1'): 32.0, ('b2', 'b2'): inf}

In [4]:
d = TrafficDemand(
    source_path=r".*(b[0-9]*)/t1",
    sink_path=r".*(b[0-9]*)/t1",
    demand=32,
    mode="combine",
)
demands = [d]
tm = TrafficManager(
    network=network,
    traffic_demands=demands,
    default_flow_policy_config=FlowPolicyConfig.SHORTEST_PATHS_ECMP,
)
tm.build_graph()
tm.expand_demands()
tm.place_all_demands()

31.999999999999904

In [5]:
d

TrafficDemand(source_path='.*(b[0-9]*)/t1', sink_path='.*(b[0-9]*)/t1', priority=0, demand=32, demand_placed=32.0, flow_policy_config=<FlowPolicyConfig.SHORTEST_PATHS_ECMP: 1>, mode='combine', attrs={}, id='.*(b[0-9]*)/t1|.*(b[0-9]*)/t1|_WaSeCXEShS7JxwxzJLm3g')

In [6]:
tm.demands

[Demand(src_node='my_clos1/b1/t1/t1-1', dst_node='my_clos1/b1/t1/t1-2', volume=0.5714285714285714, demand_class=0, flow_policy=<ngraph.lib.flow_policy.FlowPolicy object at 0x10682b380>, placed_demand=0.5714285714285714),
 Demand(src_node='my_clos1/b1/t1/t1-1', dst_node='my_clos1/b1/t1/t1-3', volume=0.5714285714285714, demand_class=0, flow_policy=<ngraph.lib.flow_policy.FlowPolicy object at 0x105fe5450>, placed_demand=0.5714285714285714),
 Demand(src_node='my_clos1/b1/t1/t1-1', dst_node='my_clos1/b1/t1/t1-4', volume=0.5714285714285714, demand_class=0, flow_policy=<ngraph.lib.flow_policy.FlowPolicy object at 0x1067eb4d0>, placed_demand=0.5714285714285714),
 Demand(src_node='my_clos1/b1/t1/t1-1', dst_node='my_clos1/b2/t1/t1-1', volume=0.5714285714285714, demand_class=0, flow_policy=<ngraph.lib.flow_policy.FlowPolicy object at 0x10612a060>, placed_demand=0.5714285714285714),
 Demand(src_node='my_clos1/b1/t1/t1-1', dst_node='my_clos1/b2/t1/t1-2', volume=0.5714285714285714, demand_class=0, f